In [1]:
import pandas as pd

import json
import glob
import os
import re

from shared import *

In [2]:
base = "../exp/"
paths = glob.glob(base + "epsilon/*.json")
data = [json.load(open(path)) for path in paths]

In [4]:
queries = pd.DataFrame.from_records([{
    **algo,
    'algo': 'IPB' if 'total_queue_pops' in algo else 'IDB',
    'epsilon': run['epsilon'],
    'graph': path_to_graph(run['args'][1]),
} for run in data for algo in run['algo_runs']])

In [26]:
queries.groupby(['epsilon', 'algo']).mean()[['length_increase_percent', 'num_iterations', 'num_forbidden_paths', 'total_exploration_time_ms', 'total_ubs_time_ms', 'running_time_ms', 'failed']]

length_increase_percent  num_iterations  num_forbidden_paths  \
epsilon algo                                                                 
0.01    IDB                  1.341602           2.820               14.738   
        IPB                  0.281126          53.020              140.568   
0.05    IDB                  0.528388           2.121                6.805   
        IPB                  0.217814          29.939               85.684   
0.10    IDB                  0.265920           1.727                2.313   
        IPB                  0.179637          19.497               58.589   
0.20    IDB                  0.180599           1.576                1.265   
        IPB                  0.150895          11.838               38.922   
0.50    IDB                  0.123584           1.217                0.368   
        IPB                  0.116355           5.040               13.920   
1.00    IDB                  0.038248           1.115                0.162   
        IPB                  0.037545           1.931                3.806   

              total_exploration_time_ms  total_ubs_time_ms  running_time_ms  \
epsilon algo                                                                  
0.01    IDB                    6.908059          85.220470        92.225261   
        IPB                   26.763573           4.036807        34.235174   
0.05    IDB                    2.556756           4.251121         6.863736   
        IPB                   13.525579           2.699696        18.151228   
0.10    IDB                    0.465995           1.107885         1.612095   
        IPB                    7.625243           1.784208        10.493443   
0.20    IDB                    0.253211           0.895844         1.183080   
        IPB                    4.985369           1.360428         7.040177   
0.50    IDB                    0.222894           0.642001         0.891090   
        IPB                    2.047124           0.957339         3.345395   
1.00    IDB                    0.209572           0.574002         0.806983   
        IPB                    0.887419           0.652318         1.714041   

              failed  
epsilon algo          
0.01    IDB    0.006  
        IPB    0.153  
0.05    IDB    0.006  
        IPB    0.074  
0.10    IDB    0.002  
        IPB    0.043  
0.20    IDB    0.001  
        IPB    0.024  
0.50    IDB    0.000  
        IPB    0.003  
1.00    IDB    0.000  
        IPB    0.000

In [15]:
queries.query("epsilon == 0.1 & algo == 'IDB'")['total_ubs_time_ms'].describe()

count    1000.000000
mean        1.107885
std         5.795070
min         0.054369
25%         0.096108
50%         0.117139
75%         0.770670
max       115.977271
Name: total_ubs_time_ms, dtype: float64

In [16]:
queries.query("epsilon == 0.01 & algo == 'IDB'")['total_ubs_time_ms'].describe()

count     1000.000000
mean        85.220470
std       2624.737451
min          0.051263
25%          0.097403
50%          0.119912
75%          0.947611
max      83002.647971
Name: total_ubs_time_ms, dtype: float64

In [17]:
queries.query("epsilon == 0.01 & algo == 'IDB'").sort_values('total_ubs_time_ms')

,failed,from,iterations,length_increase_percent,num_forbidden_paths,num_iterations,running_time_ms,to,total_exploration_time_ms,total_ubs_time_ms,algo,epsilon,graph,num_labels_pushed,total_queue_pops
4160,False,464153,"[{'algo': 'Virtual Topocore Component Query', ...",0.000000,0,1,0.150352,2773,0.084663,0.051263,IDB,0.01,OSM Europe,NaN,NaN
4555,False,669571,"[{'algo': 'Virtual Topocore Component Query', ...",0.000000,0,1,0.165958,222855,0.095549,0.052698,IDB,0.01,OSM Europe,NaN,NaN
4667,False,11648,"[{'algo': 'Virtual Topocore Component Query', ...",0.000000,0,1,0.182790,19792,0.106829,0.056329,IDB,0.01,OSM Europe,NaN,NaN
4737,False,1088654,"[{'algo': 'Virtual Topocore Component Query', ...",0.000000,0,1,0.199917,208371,0.126172,0.056519,IDB,0.01,OSM Europe,NaN,NaN
4853,False,2497605,"[{'algo': 'Virtual Topocore Component Query', ...",0.000000,0,1,0.195186,431166,0.115541,0.059145,IDB,0.01,OSM Europe,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4843,True,2443619,"[{'algo': 'Virtual Topocore Component Query', ...",68.047401,368,38,498.035811,45535,403.079790,93.120924,IDB,0.01,OSM Europe,NaN,NaN
4335,True,2372699,"[{'algo': 'Virtual Topocore Component Query', ...",0.646140,256,251,136.782331,119560,13.079410,118.539541,IDB,0.01,OSM Europe,NaN,NaN
4144,True,611724,"[{'algo': 'Virtual Topocore Component Query', ...",48.394055,340,38,471.310980,20317,343.276842,126.235464,IDB,0.01,OSM Europe,NaN,NaN
4087,True,24004,"[{'algo': 'Virtual Topocore Component Query', ...",5.012760,569,251,386.837133,931,56.763020,324.424582,IDB,0.01,OSM Europe,NaN,NaN


In [20]:
queries.query("~failed").groupby(['epsilon', 'algo']).mean()[['length_increase_percent', 'num_iterations', 'num_forbidden_paths', 'total_exploration_time_ms', 'total_ubs_time_ms', 'running_time_ms']]

length_increase_percent  num_iterations  num_forbidden_paths  \
epsilon algo                                                                 
0.01    IDB                  0.517217        1.964789             4.082495   
        IPB                  0.206281       17.257379            52.036600   
0.05    IDB                  0.347053        1.621730             2.439638   
        IPB                  0.167898       12.273218            36.285097   
0.10    IDB                  0.255438        1.457916             1.614228   
        IPB                  0.148408        9.095089            28.197492   
0.20    IDB                  0.180133        1.326326             1.014014   
        IPB                  0.130859        5.956967            20.809426   
0.50    IDB                  0.123584        1.217000             0.368000   
        IPB                  0.114966        4.299900            12.180542   
1.00    IDB                  0.038248        1.115000             0.162000   
        IPB                  0.037545        1.931000             3.806000   

              total_exploration_time_ms  total_ubs_time_ms  running_time_ms  
epsilon algo                                                                 
0.01    IDB                    0.404003           1.487244         1.943299  
        IPB                    7.550706           1.478367        10.198740  
0.05    IDB                    0.313946           1.065692         1.417990  
        IPB                    5.923629           1.493361         8.305434  
0.10    IDB                    0.268588           0.901170         1.202564  
        IPB                    3.545677           1.120898         5.231629  
0.20    IDB                    0.240795           0.780509         1.050442  
        IPB                    2.404651           0.912888         3.717233  
0.50    IDB                    0.222894           0.642001         0.891090  
        IPB                    1.768074           0.899258         2.971022  
1.00    IDB                    0.209572           0.574002         0.806983  
        IPB                    0.887419           0.652318         1.714041